In [7]:
import numpy as np
import pandas as pd


path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')

In [8]:
dataset = pd.read_csv(path + 'train_data.csv', parse_dates=["Timestamp"])
diff = dataset.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
train['solve_time'] = diff['Timestamp']

In [9]:
train.to_csv('/opt/ml/input/data/FE/FE_solve_time_train.csv', index=False)

In [10]:
dataset = pd.read_csv(path + 'test_data.csv', parse_dates=["Timestamp"])
diff = dataset.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
test['solve_time'] = diff['Timestamp']

In [11]:
test.to_csv('/opt/ml/input/data/FE/FE_solve_time_test.csv', index=False)